In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import mutual_info_score, silhouette_score
from sklearn.preprocessing import StandardScaler

In [2]:
pwd

'/Users/mac/Desktop/MCGILL/RS'

In [3]:
# Load the dataset with active users
Active_users = pd.read_csv("/Users/mac/Desktop/MCGILL/RS/practical1_2_data/active_users_from_Australia.csv")

In [4]:
Path_to_daily= "/Users/mac/Desktop/MCGILL/RS/practical1_2_data/daily_sentiments/"

In [5]:
d0301 = pd.read_csv("/Users/mac/Desktop/MCGILL/RS/practical1_2_data/daily_sentiments/Users_from_Australia_information_2020-03-01.csv")

In [6]:
d0301.tail()

,Unnamed: 0,user_id,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment
1302,1302,712899526160359424,0.336,0.726,0.546,0.233,0.602,-1
1303,1303,74735496,0.454,0.413,0.426,0.314,0.377,-1
1304,1304,3266831552,0.534,0.217,0.201,0.325,0.237,1
1305,1305,18045477,0.509,0.430,0.299,0.290,0.316,0
1306,1306,836105374717509632,0.399,0.502,0.481,0.233,0.391,-1


In [7]:
Active_users.rename(columns={'Australia': 'user_id'}, inplace=True)
Active_users.head()

,Unnamed: 0,user_id
0,0,2768501
1,1,1092378031
2,2,74382140
3,3,606571099
4,4,16675569


## We want to anticipate on the sentiment users will present the last five days.     

# 1. 
Using all days (except the last five days), you are asked to perform a Kmeans and the spectral clustering with K = 3. Color the 3 groups with respect to the given labels. Using the mutual information and the silhouette score you are asked to evaluate the quality of your clusterings. Make an interpretation of your results.

In [8]:
# Load the daily intensity dataset
daily_intensity_data = pd.DataFrame()
date_range = pd.date_range('2020-03-01', '2021-08-31', freq='D')
for date in date_range:
    filename = "Users_from_Australia_information_"+ date.strftime('%Y-%m-%d') + '.csv'
    daily_data = pd.read_csv(Path_to_daily + filename)
    daily_data['date'] = date.date()  # Add the date column
    daily_intensity_data = pd.concat([daily_intensity_data, daily_data])

In [9]:
daily_intensity_data.tail(5)

,Unnamed: 0,user_id,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,date
5998,5998,1213294041489526784,0.330,0.573,0.607,0.161,0.497,-1,2021-08-31
5999,5999,35804738,0.673,0.245,0.227,0.462,0.242,1,2021-08-31
6000,6000,1043115507862269953,0.588,0.365,0.298,0.382,0.291,1,2021-08-31
6001,6001,1427389068694855680,0.294,0.648,0.589,0.121,0.588,-1,2021-08-31
6002,6002,429422244,0.312,0.619,0.511,0.131,0.594,-1,2021-08-31


In [10]:
# Merge the datasets based on user_id
merged_data = pd.merge(Active_users, daily_intensity_data, on='user_id', how='inner')

In [11]:
# Drop Unnamed
#merged_data = merged_data.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1, inplace=True)
#merged_data.head()

In [27]:
last_five_days = date_range[-5:]
merged_data['date'] = pd.to_datetime(merged_data['date'])
last_five_days_data = merged_data[merged_data['date'].isin(last_five_days)]

#last_five_days_data.to_csv('last_five_days.csv', index=False)


In [28]:
df5days = pd.read_csv('last_five_days.csv')
df5days

,Unnamed: 0_x,user_id,Unnamed: 0_y,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,date
0,0,2768501,760,0.457870,0.492783,0.412000,0.293304,0.411565,0,2021-08-27
1,0,2768501,1976,0.429682,0.527045,0.426409,0.262682,0.437045,-1,2021-08-28
2,0,2768501,375,0.437091,0.523227,0.430682,0.259318,0.438636,-1,2021-08-29
3,0,2768501,2083,0.474741,0.486630,0.417407,0.305000,0.401407,0,2021-08-30
4,0,2768501,1663,0.458600,0.488200,0.413900,0.274967,0.415900,0,2021-08-31
...,...,...,...,...,...,...,...,...,...,...
524,113,28970958,3757,0.300667,0.479000,0.610000,0.168667,0.502667,-1,2021-08-27
525,113,28970958,626,0.269500,0.524500,0.685000,0.157500,0.531000,-1,2021-08-28
526,113,28970958,1527,0.382500,0.608500,0.559000,0.173500,0.530500,-1,2021-08-29
527,113,28970958,1385,0.471500,0.512000,0.428000,0.258000,0.479000,0,2021-08-30


In [17]:
# Remove the last five days from the merged data
last_five_days = date_range[-5:]
merged_data = merged_data[~merged_data['date'].isin(last_five_days)]

In [22]:
merged_data.shape

(53033, 10)

In [17]:
# merged_data.to_csv("merged_data.csv", index=False)

We save the merged data here cause the following steps I decide to move to google colab.    I could use the gpu there to run the Kmeans.    
Unfortunately,the memory on google colab still not enough, in this case, I move to MiniBatchKMeans to see if it could happens.
